In [2]:
import pandas as pd
import numpy as np

In [3]:
#shape of train_info
train_df = pd.read_csv("train_wInfo.csv")
display(train_df)
display(train_df.shape)

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,sar_flag
0,171189,0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4,0
1,171202,0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2,0
2,171599,0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4,0
3,171737,0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4,0
4,171142,0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3,0
...,...,...,...,...,...,...,...,...
23901,352132,364,c49b33d1fde790ec03584672903f296e486615adfdc989...,1,19.0,3218731.0,3,0
23902,352125,364,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,3,19.0,928963.0,3,0
23903,352080,364,1a93961c8fa830a1f32f5316b44f4964d65397f0311b11...,2,9.0,69080.0,6,0
23904,352075,364,9586b80d3ad1d74c0a1efe792cae2ecd073243fb01ed33...,3,3.0,262604.0,6,0


(23906, 8)

In [4]:
#shape of x_train_wInfo
x_train_df = pd.read_csv("x_train_wInfo.csv")
display(x_train_df)

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE
0,171189,0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4
1,171202,0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2
2,171599,0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4
3,171737,0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4
4,171142,0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3
...,...,...,...,...,...,...,...
23901,352132,364,c49b33d1fde790ec03584672903f296e486615adfdc989...,1,19.0,3218731.0,3
23902,352125,364,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,3,19.0,928963.0,3
23903,352080,364,1a93961c8fa830a1f32f5316b44f4964d65397f0311b11...,2,9.0,69080.0,6
23904,352075,364,9586b80d3ad1d74c0a1efe792cae2ecd073243fb01ed33...,3,3.0,262604.0,6


In [5]:
#create frequency data
#ccba dataset
#先整理成以客戶名為key的字典 所以會需要用到custinfo的資料集

#loading dataset
#custinfo: 用戶出現alert key當下時的財務狀況
custinfo = pd.read_csv("public_train_x_custinfo_full_hashed.csv")
#ccba: 用戶的信用卡狀況
ccba = pd.read_csv("public_train_x_ccba_full_hashed.csv")
#cdtx: 用戶消費狀況(日期、金額、國別)
cdtx = pd.read_csv("public_train_x_cdtx0001_full_hashed.csv")
#dp: 應該是轉帳相關
dp = pd.read_csv("public_train_x_dp_full_hashed.csv")
#remit: 外匯相關
remit = pd.read_csv("public_train_x_remit1_full_hashed.csv")


#可以看到每個客戶在每個資料集出現幾次
def get_custid_dict(df):
    dup_custid_arr = df["cust_id"].values
    custid_dict = {}
    for custid in dup_custid_arr:
        if custid in custid_dict:
            custid_dict[custid] += 1
        else:
            custid_dict[custid] = 1
    return custid_dict

#e.g. custid: (custinfo:5,ccba:4,cdtx:15,dp:0,remit:8)
def get_all_dataset_frequency(cid_list):
    dataset_num = len(cid_list)
    #extract custinfo_cid
    custinfo_cid = cid_list[0]
    column_name = ["cust_id","custinfo","ccba","cdtx","dp","remit"]
    
    cid_freq_list = []
    for cid in custinfo_cid:
        temp_list = [cid]
        for data in cid_list:
            if cid in data:
                temp_list.append(data[cid])
            else:
                temp_list.append(0)
        cid_freq_list.append(temp_list)
    
    temp_df = pd.DataFrame(cid_freq_list,columns=column_name)
    return temp_df

custinfo_cid = get_custid_dict(custinfo)
ccba_cid = get_custid_dict(ccba)
cdtx_cid = get_custid_dict(cdtx)
dp_cid = get_custid_dict(dp)
remit_cid = get_custid_dict(remit)

cid_list = [custinfo_cid,ccba_cid,cdtx_cid,dp_cid,remit_cid]
freq_cust = get_all_dataset_frequency(cid_list)
display(freq_cust)

,cust_id,custinfo,ccba,cdtx,dp,remit
0,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,23,0,0,4009,0
1,b212d14cb35676926682b2cf849e295d948888f556c07e...,56,13,8,390,63
2,e5b0002791c7852644a2730abeaa893cdf14a072ef7812...,1,0,0,55,0
3,74214c478dc6519fbefe4bc31693865bdcd698ab974b64...,53,13,24,980,3
4,0340e7611f0d82c3cb87e6194fa14bb2ccf8afbf1b3418...,2,0,0,7,1
...,...,...,...,...,...,...
7703,2a6c462095ed3a74cb1d5828ca65315cb5ef30a8b1ea54...,1,13,151,134,0
7704,830b22ee65eef58b2d51d20143f7aeb1ef9bd00b8f0435...,1,0,0,451,0
7705,f0d7fb934f97688f16691d9aeb6db22db08e8f8a07e122...,1,0,0,73,0
7706,c71eff54ff8d901590eef33bc16dfddd93a2051d7acfe4...,1,13,310,21,0


In [6]:
#concat frequency data with custinfo data
public_cust_info = pd.read_csv("public_train_x_custinfo_full_hashed.csv")
public_custinfo_wFrequency = public_cust_info.merge(freq_cust,on="cust_id")
display(public_custinfo_wFrequency)

,alert_key,cust_id,risk_rank,occupation_code,total_asset,AGE,custinfo,ccba,cdtx,dp,remit
0,352249,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,1,19.0,1465816.0,7,23,0,0,4009,0
1,356565,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,1,19.0,1465816.0,7,23,0,0,4009,0
2,357209,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,1,19.0,1465816.0,7,23,0,0,4009,0
3,358001,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,1,19.0,1465816.0,7,23,0,0,4009,0
4,359220,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,1,19.0,1465816.0,7,23,0,0,4009,0
...,...,...,...,...,...,...,...,...,...,...,...
25746,351904,2a6c462095ed3a74cb1d5828ca65315cb5ef30a8b1ea54...,1,12.0,1454.0,5,1,13,151,134,0
25747,351926,830b22ee65eef58b2d51d20143f7aeb1ef9bd00b8f0435...,1,9.0,87564.0,4,1,0,0,451,0
25748,351929,f0d7fb934f97688f16691d9aeb6db22db08e8f8a07e122...,1,4.0,1199.0,3,1,0,0,73,0
25749,352103,c71eff54ff8d901590eef33bc16dfddd93a2051d7acfe4...,1,19.0,441617.0,4,1,13,310,21,0


In [7]:
#divide into train data and test data
train_date = pd.read_csv("train_x_alert_date.csv")
test_date = pd.read_csv("public_x_alert_date.csv")
x_train = train_date.merge(public_custinfo_wFrequency,on="alert_key")
x_test = test_date.merge(public_custinfo_wFrequency,on="alert_key")

In [8]:
#concat ans to train dataset:
y_train = pd.read_csv("train_y_answer.csv")
train_dataset = x_train.merge(y_train,on="alert_key")
display(train_dataset)

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,custinfo,ccba,cdtx,dp,remit,sar_flag
0,171189,0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4,4,13,568,284,0,0
1,171202,0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2,60,0,0,217,0,0
2,171599,0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4,2,0,0,205,0,0
3,171737,0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4,8,2,3,504,0,0
4,171142,0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3,92,13,19,25183,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,352132,364,c49b33d1fde790ec03584672903f296e486615adfdc989...,1,19.0,3218731.0,3,5,13,3257,157,1,0
23902,352125,364,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,3,19.0,928963.0,3,2,0,0,20,1,0
23903,352080,364,1a93961c8fa830a1f32f5316b44f4964d65397f0311b11...,2,9.0,69080.0,6,7,0,0,71,12,0
23904,352075,364,9586b80d3ad1d74c0a1efe792cae2ecd073243fb01ed33...,3,3.0,262604.0,6,11,13,0,474,0,0


In [9]:
#customer id to integer #testing
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
cust_id_arr = train_dataset["cust_id"]
cust_id_int = le.fit_transform(cust_id_arr)
print(cust_id_int)

[4288 4592 1473 ...  766 4239 3546]


In [11]:
from xgboost import XGBClassifier

In [29]:
#with vs without cust_id

from sklearn.model_selection import train_test_split as tts 
from sklearn.metrics import classification_report
#compare with cust id and wo cust id:
x = train_dataset.drop(columns=["sar_flag","alert_key"])
x.fillna(0,inplace=True)
y = train_dataset["sar_flag"].values
x["cust_id"] = cust_id_int

x_woCust_id = x.drop(columns=["cust_id"])

X_train, X_test, y_train, y_test = tts(x,y,test_size=0.2,random_state=42)
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
y_pred_prob = xgb.predict_proba(X_test)
print("with cust id as integer")
print(classification_report(y_test,y_pred))


X_train, X_test, y_train, y_test = tts(x_woCust_id,y,test_size=0.2,random_state=42)
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
y_pred_prob = xgb.predict_proba(X_test)
print("without cust id as integer")
print(classification_report(y_test,y_pred))


d:\anaconda\envs\py36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:13:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
with cust id as integer
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      4724
           1       0.56      0.16      0.24        58

    accuracy                           0.99      4782
   macro avg       0.78      0.58      0.62      4782
weighted avg       0.98      0.99      0.98      4782

[17:13:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


d:\anaconda\envs\py36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


without cust id as integer
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      4724
           1       0.71      0.17      0.28        58

    accuracy                           0.99      4782
   macro avg       0.85      0.59      0.64      4782
weighted avg       0.99      0.99      0.99      4782



In [31]:
#oversampling vs not oversampling
from imblearn.over_sampling import SMOTE

X_train, X_test, y_train, y_test = tts(x_woCust_id,y,test_size=0.2,random_state=42)
X_resampled, y_resampled = SMOTE(sampling_strategy=0.5,random_state=42).fit_resample(X_train, y_train)
xgb = XGBClassifier()
xgb.fit(X_resampled, y_resampled)
y_pred = xgb.predict(X_test)
y_pred_prob = xgb.predict_proba(X_test)
print("without cust id as integer")
print(classification_report(y_test,y_pred))

d:\anaconda\envs\py36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:13:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
without cust id as integer
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      4724
           1       0.52      0.28      0.36        58

    accuracy                           0.99      4782
   macro avg       0.75      0.64      0.68      4782
weighted avg       0.99      0.99      0.99      4782



In [48]:
#divide into train data and test data
train_date = pd.read_csv("train_x_alert_date.csv")
test_date = pd.read_csv("public_x_alert_date.csv")
x_train = train_date.merge(public_custinfo_wFrequency,on="alert_key")
x_test = test_date.merge(public_custinfo_wFrequency,on="alert_key")
display(x_test)
#concat ans to train dataset:
y_train = pd.read_csv("train_y_answer.csv")
train_dataset = x_train.merge(y_train,on="alert_key")


xgb = XGBClassifier()

xgb_x_train = train_dataset.drop(columns=["sar_flag","cust_id","alert_key","date"])
display(xgb_x_train)
xgb_y_train = train_dataset["sar_flag"]
display(xgb_y_train)
xgb_x_train = xgb_x_train.fillna(0)

X_resampled, y_resampled = SMOTE(sampling_strategy=0.5,random_state=42).fit_resample(xgb_x_train, xgb_y_train)
print(X_resampled.shape)


xgb_x_test = x_test.drop(columns=["cust_id","alert_key","date"])
display(xgb_x_test.shape)


xgb.fit(X_resampled, y_resampled)
ans = xgb.predict_proba(xgb_x_test)
ans = ans[:,1]


ans_csv = x_test[["alert_key"]]
ans_csv["probability"] = ans
display(ans_csv)
ans_csv.to_csv("ans_csv1204.csv",index=False)

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,custinfo,ccba,cdtx,dp,remit
0,352342,365,7629f010080267f680a4ca9c6cac71c4d4d86933dcc7ed...,1,17.0,506366.0,3,48,13,86,864,56
1,352866,365,e50deb3f2f87bf45e04940303f80c96929c836b0b03f54...,1,12.0,309830.0,4,14,13,0,55,0
2,352696,365,c6def618ad861703c025be4f41bdf7569310228ae93aef...,1,2.0,444392.0,3,34,13,209,339,47
3,352330,365,96d70b798f211393cfbcb83a8acd19f3e8c9c5c205960a...,1,19.0,143930.0,4,1,0,0,131,0
4,352683,365,40fa92acb89d1ebb319f0f200d31a5fcbdfcbd8ad432d7...,3,17.0,2112.0,2,17,0,0,333,26
...,...,...,...,...,...,...,...,...,...,...,...,...
1840,364472,393,bdda3ef79c5f052fd281c9f28f50a25977444eceaf9751...,3,19.0,2390889.0,4,36,13,221,760,40
1841,364788,393,0ef49d08b4af68e9cb2b2aa63198699ee7dc7b5e1c4550...,2,19.0,28681.0,2,1,0,0,155,2
1842,364673,393,f8ce663f765e2459578d285a032a16046fba0b2bba8e12...,1,19.0,213486.0,6,3,0,0,339,32
1843,364626,393,1729de666fbfd3bd42133e8226f5ade5709ef141d8e0b7...,1,19.0,2913.0,2,4,0,0,182,0


,risk_rank,occupation_code,total_asset,AGE,custinfo,ccba,cdtx,dp,remit
0,1,17.0,375576.0,4,4,13,568,284,0
1,3,12.0,2717416.0,2,60,0,0,217,0
2,1,12.0,326517.0,4,2,0,0,205,0
3,3,14.0,1014759.0,4,8,2,3,504,0
4,3,12.0,241719.0,3,92,13,19,25183,0
...,...,...,...,...,...,...,...,...,...
23901,1,19.0,3218731.0,3,5,13,3257,157,1
23902,3,19.0,928963.0,3,2,0,0,20,1
23903,2,9.0,69080.0,6,7,0,0,71,12
23904,3,3.0,262604.0,6,11,13,0,474,0


0        0
1        0
2        0
3        0
4        0
        ..
23901    0
23902    0
23903    0
23904    0
23905    0
Name: sar_flag, Length: 23906, dtype: int64

(35508, 9)


(1845, 9)

[17:24:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


d:\anaconda\envs\py36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
d:\anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,alert_key,probability
0,352342,0.000071
1,352866,0.000038
2,352696,0.000607
3,352330,0.015692
4,352683,0.000326
...,...,...
1840,364472,0.000075
1841,364788,0.005765
1842,364673,0.010156
1843,364626,0.007341


In [64]:
handout_data = pd.read_csv("D:\Python_projects\DataMining\yushan\預測的案件名單及提交檔案範例.csv")
handout_data = handout_data.merge(ans_csv,on="alert_key",how="left")
display(handout_data)
handout_data =  handout_data.drop(columns="probability_x")
handout_data = handout_data.rename(columns={"probability_y":"probability"})
handout_data = handout_data.fillna(0)
display(handout_data)
handout_data.to_csv("handout_dataVer1.csv",index=False)



,alert_key,probability_x,probability_y
0,357307,0.000017,0.003414
1,376329,0.000324,NaN
2,373644,0.000372,NaN
3,357668,0.000489,0.003072
4,354443,0.000526,0.000607
...,...,...,...
3845,364485,0.997702,0.052611
3846,363155,0.998987,0.000062
3847,368710,0.999694,NaN
3848,358067,0.999821,0.000015


,alert_key,probability
0,357307,0.003414
1,376329,0.000000
2,373644,0.000000
3,357668,0.003072
4,354443,0.000607
...,...,...
3845,364485,0.052611
3846,363155,0.000062
3847,368710,0.000000
3848,358067,0.000015
